In [7]:
# Dependencies and Setup
# render the figure in a notebook instead of displaying a dump of the figure object.
%matplotlib inline  
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import sem
# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

#mouse data : Mouse ID	Drug
mouseData_df= pd.read_csv("data/mouse_drug_data.csv")
#trial data: Mouse ID	Timepoint	Tumor Volume (mm3)	Metastatic Sites
trialData_df= pd.read_csv("data/clinicaltrial_data.csv")

In [8]:
allData_df=pd.merge(mouseData_df,trialData_df, on=["Mouse ID","Mouse ID"])
dtgroup=allData_df.groupby(['Drug','Timepoint'])
errdf=dtgroup['Tumor Volume (mm3)'].agg([np.mean, sem]).reset_index()
errdf.head()

,Drug,Timepoint,mean,sem
0,Capomulin,0,45.000000,0.000000
1,Capomulin,5,44.266086,0.448593
2,Capomulin,10,43.084291,0.702684
3,Capomulin,15,42.064317,0.838617
4,Capomulin,20,40.716325,0.909731


In [9]:
meandf=errdf.drop(columns=['sem']) 
tmeandf=meandf.set_index('Timepoint')

In [10]:
tmeandf.head()


,Drug,mean
Timepoint,,
0,Capomulin,45.000000
5,Capomulin,44.266086
10,Capomulin,43.084291
15,Capomulin,42.064317
20,Capomulin,40.716325


In [14]:
pdf=tmeandf.pivot(columns='Drug', values='mean')
pdf.head()

Drug,Capomulin,Ceftamin,Infubinol,Ketapril,Naftisol,Placebo,Propriva,Ramicane,Stelasyn,Zoniferol
Timepoint,,,,,,,,,,
0,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000
5,44.266086,46.503051,47.062001,47.389175,46.796098,47.125589,47.248967,43.944859,47.527452,46.851818
10,43.084291,48.285125,49.403909,49.582269,48.694210,49.423329,49.101541,42.531957,49.463844,48.689881
15,42.064317,50.094055,51.296397,52.399974,50.933018,51.359742,51.067318,41.495061,51.529409,50.779059
20,40.716325,52.157049,53.197691,54.920935,53.644087,54.364417,53.346737,40.238325,54.067395,53.170334
